In [3]:
# Colab에서 필요한 라이브러리 설치
pip install mediapipe opencv-python numpy openai python-dotenv
apt-get update && apt-get install -y ffmpeg

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [4]:
# 2. 라이브러리 임포트 및 OpenAI API 키 설정
import openai
from openai import OpenAI # import openai
import cv2
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import io
import json
import ast
import base64
from dotenv import load_dotenv
from pathlib import Path
import os
import time
import math

In [5]:
# 비디오에서 특정 프레임을 추출하는 함수
def read_video_opencv(video_path, frame_indices):
    """
    OpenCV를 사용하여 비디오에서 특정 프레임들을 추출합니다.

    Parameters:
    - video_path: 비디오 파일 경로
    - frame_indices: 추출할 프레임의 인덱스 리스트

    Returns:
    - 추출된 프레임들의 리스트 (NumPy 배열 형식) 또는 None
    """
    try:
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"비디오 파일을 열 수 없습니다: {video_path}")
            return None

        frames = []
        frame_counter = 0
        frame_indices_set = set(frame_indices)

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            if frame_counter in frame_indices_set:
                frames.append(frame)
                if len(frames) == len(frame_indices):
                    break

            frame_counter += 1

        cap.release()

        if not frames:
            print("지정된 프레임 인덱스에 해당하는 프레임을 찾을 수 없습니다.")
            return None

        return frames

    except Exception as e:
        print(f"비디오에서 프레임 추출 중 오류 발생: {e}")
        return None

In [6]:
# 비디오 길이 가져오기 함수
def get_video_duration(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Could not open video file: {video_path}")
        return None
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    duration = total_frames / fps
    cap.release()
    return duration

In [7]:
# 비디오에서 프레임 추출 함수
def download_and_sample_video_local(video_path, start_time=0, duration=60, frame_interval=3):
    """
    주어진 비디오 파일에서 지정된 시작 시간과 지속 시간 내에서 일정 간격으로 프레임을 추출합니다.

    Parameters:
    - video_path: 비디오 파일 경로
    - start_time: 추출 시작 시간 (초 단위)
    - duration: 추출할 구간의 길이 (초 단위)
    - frame_interval: 프레임 추출 간격 (초 단위)

    Returns:
    - 추출된 프레임들의 리스트 (NumPy 배열 형식) 또는 None
    """
    try:
        # OpenCV로 비디오 캡처
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"비디오 파일을 열 수 없습니다: {video_path}")
            return None

        fps = cap.get(cv2.CAP_PROP_FPS)
        if fps == 0:
            fps = 30.0  # 기본 FPS 설정
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        # 추출할 프레임 인덱스 계산
        start_frame = int(start_time * fps)
        end_frame = int((start_time + duration) * fps)
        frame_indices = list(range(start_frame, end_frame, int(frame_interval * fps)))

        frames = []
        frame_counter = 0

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            if frame_counter in frame_indices:
                frames.append(frame)
                if len(frames) == len(frame_indices):
                    break

            frame_counter += 1

        cap.release()

        if not frames:
            print("지정된 프레임 인덱스에 해당하는 프레임을 찾을 수 없습니다.")
            return None

        return np.array(frames)

    except Exception as e:
        print(f"비디오에서 프레임 추출 중 오류 발생: {e}")
        return None

### 추가 함수

In [27]:
import os
import cv2
import mediapipe as mp
import numpy as np
from typing import List, Dict

# Mediapipe 초기화 (전역 한 번)
mp_pose = mp.solutions.pose
mp_face = mp.solutions.face_mesh
mp_hands = mp.solutions.hands

pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
face_mesh = mp_face.FaceMesh(static_image_mode=False, max_num_faces=1, min_detection_confidence=0.5, min_tracking_confidence=0.5)
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.3, min_tracking_confidence=0.3)

In [9]:
def calculate_lack_of_eye_contact_score(face_landmarks, frame_width):
    """
    발표자의 시선 부족 정도를 점수로 반환합니다.

    Args:
        face_landmarks: Mediapipe FaceMesh 랜드마크.
        frame_width: 프레임 너비.

    Returns:
        float: 시선 부족 점수 (0에서 1 사이).
    """
    LEFT_EYE = 33
    RIGHT_EYE = 263

    # 왼쪽/오른쪽 눈의 x좌표 구하기
    left_eye_x = face_landmarks.landmark[LEFT_EYE].x * frame_width
    right_eye_x = face_landmarks.landmark[RIGHT_EYE].x * frame_width

    # 화면 중앙 범위 설정
    center_min = 0.4 * frame_width
    center_max = 0.6 * frame_width

    # 눈 위치가 화면 중앙에서 얼마나 벗어났는지 계산
    left_deviation = max(center_min - left_eye_x, left_eye_x - center_max, 0)
    right_deviation = max(center_min - right_eye_x, right_eye_x - center_max, 0)

    # 편차를 정규화하여 점수 환산
    gaze_score = (left_deviation + right_deviation) / frame_width
    return round(min(gaze_score / 0.1, 1.0), 2)  # 소수점 두 자리로 제한

In [10]:
def calculate_excessive_gestures_score(hand_landmarks):
    """
    과도한 손동작 정도를 점수로 반환합니다. (엄지와 검지 끝 랜드마크 거리로 예측)

    Args:
        hand_landmarks: Mediapipe Hands 랜드마크.

    Returns:
        float: 과도한 제스처 점수 (0에서 1 사이).
    """
    THUMB_TIP = mp_hands.HandLandmark.THUMB_TIP.value
    INDEX_TIP = mp_hands.HandLandmark.INDEX_FINGER_TIP.value

    # 엄지 끝, 검지 끝 랜드마크 좌표
    thumb_tip = hand_landmarks.landmark[THUMB_TIP]
    index_tip = hand_landmarks.landmark[INDEX_TIP]

    # 두 점 사이의 거리
    distance = np.sqrt((thumb_tip.x - index_tip.x) ** 2 + (thumb_tip.y - index_tip.y) ** 2)

    # 0.2를 기준으로 1까지 정규화
    return round(min(distance / 0.2, 1.0), 2)  # 소수점 두 자리로 제한

In [11]:
def calculate_hand_movement_score(current_hand_landmarks, previous_hand_landmarks, threshold=0.1):
    """
    현재 프레임과 이전 프레임에서 손목 위치 변화로 움직임 점수를 계산합니다.

    Args:
        current_hand_landmarks: 현재 프레임의 Mediapipe HandLandmarks 객체.
        previous_hand_landmarks: 이전 프레임의 Mediapipe HandLandmarks 객체.
        threshold: 움직임 점수 계산 기준이 되는 임계값.

    Returns:
        float: 움직임 점수 (0에서 1 사이).
    """
    if previous_hand_landmarks is None:
        # 이전 프레임 정보가 없으면 움직임 점수를 0으로 반환
        return 0.0

    WRIST = mp_hands.HandLandmark.WRIST.value
    current_wrist = current_hand_landmarks.landmark[WRIST]
    previous_wrist = previous_hand_landmarks.landmark[WRIST]

    # 손목의 이동 거리 계산
    movement_distance = np.sqrt((current_wrist.x - previous_wrist.x)**2 + (current_wrist.y - previous_wrist.y)**2)
    # threshold 대비 비율로 점수 환산
    movement_score = min(movement_distance / threshold, 1.0)
    return round(movement_score, 2)  # 소수점 두 자리로 제한

In [12]:
def is_hand_out_of_frame(hand_landmarks, frame_width, frame_height):
    """
    손이 화면 밖에 위치하는지 여부를 판단합니다.

    Args:
        hand_landmarks: Mediapipe HandLandmarks 객체.
        frame_width: 프레임의 너비.
        frame_height: 프레임의 높이.

    Returns:
        bool: 손이 화면 밖이면 True, 아니면 False.
    """
    for landmark in hand_landmarks.landmark:
        x, y = landmark.x * frame_width, landmark.y * frame_height
        if x < 0 or x > frame_width or y < 0 or y > frame_height:
            return True
    return False

In [13]:
def calculate_head_position_score(pose_landmarks, frame_width, frame_height):
    """
    머리(코 위치)의 X, Y 편차로 자세 점수를 계산합니다.

    Args:
        pose_landmarks: Mediapipe Pose 랜드마크.
        frame_width: 프레임 너비.
        frame_height: 프레임 높이.

    Returns:
        float: 자세 점수 (0: 좋음 ~ 1: 나쁨).
    """
    NOSE = mp_pose.PoseLandmark.NOSE.value
    nose_x = pose_landmarks.landmark[NOSE].x * frame_width
    nose_y = pose_landmarks.landmark[NOSE].y * frame_height

    # 화면 중심
    center_x, center_y = frame_width / 2, frame_height / 2

    # X, Y 축 편차 계산
    x_distance = abs(nose_x - center_x)
    y_distance = abs(nose_y - center_y)

    # 최대 허용 거리(정상 자세 범위)
    max_x_distance = frame_width * 0.1
    max_y_distance = frame_height * 0.2

    # X, Y 편차를 정규화하여 점수 계산
    x_score = min(x_distance / max_x_distance, 1.0)
    y_score = min(y_distance / max_y_distance, 1.0)

    # X, Y 평균
    posture_score = (x_score + y_score) / 2
    return round(posture_score, 2)  # 소수점 두 자리로 제한

In [14]:
def calculate_sudden_movement_score(pose_landmarks, previous_pose_landmarks, threshold=0.1):
    """
    갑작스러운 움직임 정도를 점수로 반환합니다.
    어깨와 엉덩이의 평균 위치 변화로 추정.

    Args:
        pose_landmarks: 현재 프레임의 포즈 랜드마크.
        previous_pose_landmarks: 이전 프레임의 포즈 랜드마크.
        threshold: 움직임 감지 임계값.

    Returns:
        float: 갑작스러운 움직임 점수 (0에서 1 사이).
    """
    if previous_pose_landmarks is None:
        return 0.0

    def get_center(landmarks, points):
        x = np.mean([landmarks.landmark[p].x for p in points])
        y = np.mean([landmarks.landmark[p].y for p in points])
        return np.array([x, y])

    # 상체 중심 (양 어깨, 양 엉덩이 평균 좌표)
    current_center = get_center(pose_landmarks, [
        mp_pose.PoseLandmark.LEFT_SHOULDER.value,
        mp_pose.PoseLandmark.RIGHT_SHOULDER.value,
        mp_pose.PoseLandmark.LEFT_HIP.value,
        mp_pose.PoseLandmark.RIGHT_HIP.value
    ])
    previous_center = get_center(previous_pose_landmarks, [
        mp_pose.PoseLandmark.LEFT_SHOULDER.value,
        mp_pose.PoseLandmark.RIGHT_SHOULDER.value,
        mp_pose.PoseLandmark.LEFT_HIP.value,
        mp_pose.PoseLandmark.RIGHT_HIP.value
    ])

    # 현재 상체 중심과 이전 상체 중심 사이의 이동 거리
    movement = np.linalg.norm(current_center - previous_center)
    return round(min(movement / threshold, 1.0), 2)

In [15]:
def draw_debug_info(frame, pose_landmarks, frame_width, frame_height, posture_score):
    """
    디버깅을 위한 시각화 함수. 코 위치와 화면 중심선을 표시하고 자세 점수를 보여줍니다.

    Args:
        frame: OpenCV 프레임.
        pose_landmarks: Mediapipe Pose 랜드마크.
        frame_width: 프레임 너비.
        frame_height: 프레임 높이.
        posture_score: 계산된 자세 점수.
    """
    NOSE = mp_pose.PoseLandmark.NOSE.value
    nose_x = int(pose_landmarks.landmark[NOSE].x * frame_width)
    nose_y = int(pose_landmarks.landmark[NOSE].y * frame_height)

    # 머리(코) 위치 표시
    cv2.circle(frame, (nose_x, nose_y), 5, (0, 255, 0), -1)

    # 화면 중심선 그리기
    center_x, center_y = frame_width // 2, frame_height // 2
    cv2.line(frame, (center_x, 0), (center_x, frame_height), (255, 0, 0), 2)
    cv2.line(frame, (0, center_y), (frame_width, center_y), (255, 0, 0), 2)

    # 자세 점수 표시
    cv2.putText(frame, f"Posture Score: {posture_score:.2f}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    # 자세가 매우 나쁘다면 추가 메시지
    if posture_score > 0.7:
        cv2.putText(frame, "Poor Posture", (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

In [16]:
def calculate_gestures_score(excessive_gestures_score, hand_movement_score):
    """
    손 관련 점수들의 평균을 계산합니다.

    Args:
        excessive_gestures_score: 과도한 제스처 점수.
        hand_movement_score: 손 움직임 점수.

    Returns:
        float: gestures 점수 (0~1 사이).
    """
    scores = [excessive_gestures_score, hand_movement_score]
    return round(sum(scores) / len(scores), 2)

In [17]:
def is_hand_raised_above_threshold(hand_landmarks, frame_height, threshold_ratio=0.7):
    """
    손이 화면 높이의 특정 비율 이상에 위치하는지 판단합니다.

    Args:
        hand_landmarks: Mediapipe HandLandmarks 객체.
        frame_height: 프레임의 높이.
        threshold_ratio: 화면 높이 비율 기준 (기본: 0.7).

    Returns:
        bool: 손이 기준 높이 이상이면 True, 아니면 False.
    """
    WRIST = mp_hands.HandLandmark.WRIST.value
    hand_height = hand_landmarks.landmark[WRIST].y * frame_height
    return hand_height < frame_height * (1 - threshold_ratio)  # 위쪽이 0에 가까움

In [18]:
def draw_text(frame, text, x, y, color=(0, 255, 0)):
    """
    프레임 위에 텍스트를 그립니다.

    Args:
        frame: OpenCV 프레임
        text: 표시할 텍스트
        x, y: 텍스트 좌표
        color: 텍스트 색상 (BGR)
    """
    cv2.putText(frame, text, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

In [22]:
def draw_bounding_boxes(frame, feedback, pose_landmarks, face_landmarks, hand_landmarks):
    """
    문제 영역에 Bounding Box를 그리고 잘못된 부분을 시각적으로 표시합니다.

    Args:
        frame: OpenCV 프레임.
        feedback: 분석된 피드백 결과 딕셔너리.
        pose_landmarks: Mediapipe Pose 랜드마크.
        face_landmarks: Mediapipe Face 랜드마크.
        hand_landmarks: Mediapipe Hand 랜드마크 리스트.

    Returns:
        frame: Bounding Box가 추가된 프레임.
    """
    frame_height, frame_width, _ = frame.shape

    # 자세 문제 표시
    if feedback["posture_score"] > 0.8:
        if pose_landmarks:
            nose = pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE]
            nose_x, nose_y = int(nose.x * frame_width), int(nose.y * frame_height)
            cv2.rectangle(frame, (nose_x - 50, nose_y - 50), (nose_x + 50, nose_y + 50), (0, 0, 255), 2)
            cv2.putText(frame, "Posture Issue", (nose_x - 70, nose_y - 60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    # 시선 문제 표시
    if feedback["gaze_score"] > 0.5:
        if face_landmarks:
            left_eye = face_landmarks.landmark[33]  # 왼쪽 눈
            right_eye = face_landmarks.landmark[263]  # 오른쪽 눈
            left_eye_x, left_eye_y = int(left_eye.x * frame_width), int(left_eye.y * frame_height)
            right_eye_x, right_eye_y = int(right_eye.x * frame_width), int(right_eye.y * frame_height)
            cv2.rectangle(frame, (left_eye_x - 10, left_eye_y - 10), (right_eye_x + 10, right_eye_y + 10), (255, 0, 0), 2)
            cv2.putText(frame, "Gaze Issue", (left_eye_x - 50, left_eye_y - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    # 손 제스처 문제 표시
    if feedback["gestures_score"] > 0.5 or feedback["hand_raised"]:
        if hand_landmarks:
            for hand in hand_landmarks:
                wrist = hand.landmark[mp_hands.HandLandmark.WRIST]
                wrist_x, wrist_y = int(wrist.x * frame_width), int(wrist.y * frame_height)
                cv2.rectangle(frame, (wrist_x - 50, wrist_y - 50), (wrist_x + 50, wrist_y + 50), (0, 255, 0), 2)
                cv2.putText(frame, "Gesture Issue", (wrist_x - 70, wrist_y - 60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # 갑작스러운 움직임 표시
    if feedback["sudden_movement_score"] > 0.5:
        cv2.putText(frame, "Sudden Movement Detected", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 165, 255), 2)

    return frame

In [23]:
def process_feedback_and_visualize(frames, feedback_results):
    """
    분석된 피드백 결과를 기반으로 잘못된 부분을 시각적으로 표시합니다.

    Args:
        frames: 비디오 프레임 리스트.
        feedback_results: 각 프레임의 피드백 결과 리스트.

    Returns:
        processed_frames: Bounding Box와 텍스트가 추가된 프레임 리스트.
    """
    processed_frames = []
    for idx, (frame, feedback) in enumerate(zip(frames, feedback_results)):
        if feedback["frame_index"] == idx:
            pose_landmarks = feedback.get("pose_landmarks", None)
            face_landmarks = feedback.get("face_landmarks", None)
            hand_landmarks = feedback.get("hand_landmarks", None)
            frame = draw_bounding_boxes(frame, feedback, pose_landmarks, face_landmarks, hand_landmarks)
            processed_frames.append(frame)
    return processed_frames

In [24]:
def analyze_frame(frame, previous_pose_landmarks=None, previous_hand_landmarks=None):
    """
    단일 프레임을 분석하여 자세, 시선, 손동작 등의 피드백 정보를 반환합니다.

    Args:
        frame: 분석할 OpenCV 프레임.
        previous_pose_landmarks: 이전 프레임의 포즈 랜드마크 - gestures.
        previous_hand_landmarks: 이전 프레임의 손 랜드마크 - gestures.

    Returns:
        feedback: 분석 결과를 담은 딕셔너리.
        current_pose_landmarks: 현재 포즈 랜드마크.
        current_hand_landmarks: 현재 손 랜드마크.
    """
    frame_height, frame_width, _ = frame.shape
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Mediapipe 결과 처리
    pose_results = pose.process(rgb_frame)
    face_results = face_mesh.process(rgb_frame)
    hand_results = hands.process(rgb_frame)

    # 초기 점수들
    posture_score = 0
    gaze_score = 0
    excessive_gestures_score = 0
    sudden_movement_score = 0
    hand_out_of_frame_flag = False
    hand_movement_score = 0
    hand_raised_flag = False  # 손이 너무 높이 올려졌는지 여부

    current_pose_landmarks = None
    current_hand_landmarks = None

    # 포즈 분석
    if pose_results.pose_landmarks:
        current_pose_landmarks = pose_results.pose_landmarks
        posture_score = calculate_head_position_score(current_pose_landmarks, frame_width, frame_height)
        sudden_movement_score = calculate_sudden_movement_score(current_pose_landmarks, previous_pose_landmarks)

    # 얼굴(시선) 분석
    if face_results.multi_face_landmarks:
        gaze_score = calculate_lack_of_eye_contact_score(face_results.multi_face_landmarks[0], frame_width)

    # 손 분석
    if hand_results.multi_hand_landmarks:
        for hl in hand_results.multi_hand_landmarks:
            # 과도한 제스처 점수 업데이트
            g_score = calculate_excessive_gestures_score(hl)
            if g_score > excessive_gestures_score:
                excessive_gestures_score = g_score

            # 손이 화면 밖으로 나갔는지 판정
            if is_hand_out_of_frame(hl, frame_width, frame_height):
                hand_out_of_frame_flag = True

            # 손이 너무 높이 올려졌는지 판정
            if is_hand_raised_above_threshold(hl, frame_height):
                hand_raised_flag = True

            # 이전 손 위치와 비교하여 손 움직임 점수 업데이트
            if previous_hand_landmarks is not None:
                m_score = calculate_hand_movement_score(hl, previous_hand_landmarks)
                if m_score > hand_movement_score:
                    hand_movement_score = m_score

        # 현재 손 랜드마크 업데이트 (첫 번째 손 기준)
        current_hand_landmarks = hand_results.multi_hand_landmarks[0]

    # gestures 평균 점수 계산
    gestures_score = calculate_gestures_score(excessive_gestures_score, hand_movement_score)

    # 피드백 딕셔너리
    feedback = {
        "posture_score": round(posture_score, 2),
        "gaze_score": round(gaze_score, 2),
        "gestures_score": gestures_score,  # gestures의 평균 점수
        "sudden_movement_score": round(sudden_movement_score, 2),
        "hand_out_of_frame": hand_out_of_frame_flag,
        "hand_raised": hand_raised_flag  # 손이 너무 높이 올려졌는지 여부
    }

    return feedback, current_pose_landmarks, current_hand_landmarks


In [25]:
def analyze_video_frames(frames, posture_threshold=0.8, gaze_threshold=0.5,
                         gesture_threshold=0.5, movement_threshold=0.5,
                         hand_movement_threshold=0.5):
    """
    여러 프레임(영상)을 순차적으로 분석하여 결과를 반환합니다.
    각 임계값을 초과하거나 특정 조건(hand_out_of_frame, hand_raised)을 만족하면 프레임과 결과를 추출합니다.

    Args:
        frames: 분석할 프레임 리스트.
        posture_threshold: 자세 불량 판정 기준 점수 - posture_body.
        gaze_threshold: 시선 부족 판정 기준 점수 - gaze_processing.
        gesture_threshold: 손 제스처 과도 판정 기준 점수 - gestures.
        movement_threshold: 갑작스런 움직임 판정 기준 점수 - movement.
        hand_movement_threshold: 손 움직임 과도 판정 기준 점수 - gestures.

    Returns:
        filtered_feedback_results: 기준 초과 또는 조건 만족 피드백 결과 리스트.
        filtered_frames: 기준 초과 또는 조건 만족 프레임 리스트.
    """
    filtered_feedback_results = []
    filtered_frames = []
    previous_pose_landmarks = None
    previous_hand_landmarks = None

    for idx, frame in enumerate(frames):
        # 각 프레임을 분석
        feedback, current_pose_landmarks, current_hand_landmarks = analyze_frame(
            frame, previous_pose_landmarks, previous_hand_landmarks
        )

        # 기준 초과 여부 또는 특정 조건 만족 여부 확인
        if (feedback["posture_score"] > posture_threshold or
            feedback["gaze_score"] > gaze_threshold or
            feedback["gestures_score"] > gesture_threshold or
            feedback["sudden_movement_score"] > movement_threshold or
            feedback["hand_out_of_frame"] or  # 손이 화면 밖으로 나간 경우
            feedback["hand_raised"]):         # 손이 너무 높이 올라간 경우

            # 기준 초과 또는 조건 만족 프레임 저장
            feedback["frame_index"] = idx
            filtered_feedback_results.append(feedback)
            filtered_frames.append(frame)

        # 이전 랜드마크 업데이트
        previous_pose_landmarks = current_pose_landmarks if current_pose_landmarks else previous_pose_landmarks
        previous_hand_landmarks = current_hand_landmarks if current_hand_landmarks else previous_hand_landmarks

    return filtered_feedback_results, filtered_frames


In [28]:
import os
import cv2

# 비디오 경로 및 출력 디렉토리 설정
video_path = "/content/pr_video_test (1).mp4"
output_dir = "output_frames"
os.makedirs(output_dir, exist_ok=True)

# 프레임 샘플링 함수 호출 (사용자 정의 함수)
frames = download_and_sample_video_local(video_path, start_time=0, duration=60, frame_interval=0.5)

if frames is not None and len(frames) > 0:
    # 비디오 프레임 분석
    feedbacks, filtered_frames = analyze_video_frames(frames)

    # 조건을 만족하는 결과 프레임 저장
    for i, frame in enumerate(filtered_frames):
        cv2.imwrite(f"{output_dir}/frame_{i}.jpg", frame)

    # 피드백 결과 출력
    print("피드백 결과:")
    for fb in feedbacks:
        print(fb)
else:
    print("프레임을 추출할 수 없습니다.")


피드백 결과:
{'posture_score': 0.63, 'gaze_score': 0.36, 'gestures_score': 0.0, 'sudden_movement_score': 1.0, 'hand_out_of_frame': False, 'hand_raised': False, 'frame_index': 23}
{'posture_score': 0.73, 'gaze_score': 0.57, 'gestures_score': 0.0, 'sudden_movement_score': 0.32, 'hand_out_of_frame': False, 'hand_raised': False, 'frame_index': 24}
{'posture_score': 0.53, 'gaze_score': 0.0, 'gestures_score': 0.42, 'sudden_movement_score': 0.91, 'hand_out_of_frame': False, 'hand_raised': False, 'frame_index': 27}
